## Samples spatio-temporal distribution and distance between sites

## Model performance comparison across factors

## When does the model work well? When does it not?

## Comparison of model confidence interval and forecasted extreme snowmelt change